<h2>Veriseti Manuel Test Uygulaması</h2>
<p>Yolo Etiket formatı yani resim boyutlarına göre scale edilmiş (label,center_x,center_y,width,height) 
değerlerini okuyup frame üzerine çizdirerek manuel test işlemi yapmak. Etiketlerin doğru mu işaretlendiğini test etmek</p>

<ul><b>Klavye Kontrolleri</b>
  <li>space -> bir sonraki görsele geçer</li>
  <li>p -> görseli problemli olarak işaretleyip bir sonraki görsele geçer</li>
  <li>q -> çıkış işlemi yapar</li>
</ul>

<p>Problemli olarak işaretlenen görsellerin isimleri dizindeki "problems.txt" içerisinde bulunur.</p>
<p><i>İşlemler için dosyalar standart dizilimde olmalıdır, bir alt hücrede dizilim yazdırılmıştır. Bu dosya detect klasörünün yanında olmalıdır altında değil.</i></p>

In [1]:
import seedir as sd
sd.seedir("./detect/")

detect/
├─detect.yaml
├─images/
│ ├─test/
│ │ ├─bookstore_video0_0.jpg
│ │ ├─bookstore_video0_30.jpg
│ │ ├─bookstore_video0_60.jpg
│ │ └─bookstore_video0_90.jpg
│ ├─train/
│ └─val/
└─labels/
  ├─test/
  │ ├─bookstore_video0_0.txt
  │ ├─bookstore_video0_30.txt
  │ ├─bookstore_video0_60.txt
  │ └─bookstore_video0_90.txt
  ├─train/
  └─val/


In [1]:
from spiral_tools.constants import label_dict,label_color_dict
from spiral_tools.utils import path_sequence,format_line

In [2]:
import cv2
import numpy as np
import os
from tqdm import tqdm
from IPython.display import clear_output
import random
from sklearn.utils import shuffle

In [3]:
# Ekran çözünürlüğüne ulaşalım
import tkinter as tk
root = tk.Tk()

screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

In [4]:
frame_display_size = (int(screen_width*0.8),int(screen_height*0.8))

color1 = (100,100,255)
color2 = (0,0,0)
color3 = (50,210,50)
color4 = (50,50,210)

font = cv2.FONT_HERSHEY_DUPLEX

is_randomized = True

## Alt Datasetlerden Seçmek ve ayarlar (./datasets altındakiler)

#### Dataset Seçme ve Ayarlar

In [5]:
ds_list = os.listdir("./datasets")
for i,ds in enumerate(ds_list):
    print(i,ds)
    
print("\nIndex girin: ")
d_i = int(input("> "))

if d_i not in range(len(ds_list)):
    print("\nuygun bir index girin")
else:
    
    dataset_name = ds_list[d_i]    
    print("\nseçilen veriseti:",dataset_name)
    
    root = f"./datasets/{ds_list[d_i]}/detect"
    print("\nroot: ",root)

    print("\nGörüntüler rastgele gelsin \nhayır-> 0 \nevet-> 1")
    is_rand = int(input("> "))
    clear_output()
    print(dataset_name)
    if is_rand == 0:
        is_randomized = False
        print("Görüntüler rastgele olmayacak")
    elif is_rand ==1:
        is_randomized = True
        print("Görüntüler rastgele olacak")
    else:
        print("Yanlış karakter girildi. Otomatik Tercih: ",is_randomized)

Visdrone_distil
Görüntüler rastgele olacak


In [6]:
images_path = path_sequence(root,"images")
labels_path = path_sequence(root,"labels")

problems = []

shown_counter = 0
problem_counter = 0
notproblem_counter = 0


for i,c in tqdm( enumerate(["train","test","val"])):
    images_file = path_sequence(images_path,c)
    labels_file = path_sequence(labels_path,c)
    print("işleniyor: ", images_file)
    print("işleniyor: ", labels_file)
    image_names = os.listdir(images_file)
    label_names = os.listdir(labels_file)
    
    if is_randomized:
        image_names, label_names = shuffle(image_names, label_names)
    
    for im_name,label_name in zip(image_names,label_names):
        ## dosyalara ulaş
        print("--",im_name,label_name)
        print("gösterildi: ",shown_counter)
        # frame
        original_image = cv2.imread(path_sequence(images_file,im_name))
        image = original_image.copy()
        
        # etiketler
        f = open(path_sequence(labels_file,label_name),"r")
        lines = f.readlines()
        
        for line in lines:
            label, norm_center_x, norm_center_y, norm_width, norm_height = format_line(line)
            
            width = image.shape[1] * norm_width
            height = image.shape[0] * norm_height
            x1 = int(image.shape[1]*norm_center_x) - int(width/2)
            x2 = int(image.shape[1]*norm_center_x) + int(width/2)
            y1 = int(image.shape[0]*norm_center_y) - int(height/2)
            y2 = int(image.shape[0]*norm_center_y) + int(height/2)
            
            # etiketleri çizdirelim
            cv2.putText(image,f"{label_dict[int(label)]}",(x1+5,y1+15),font,0.55,label_color_dict[int(label)])
            cv2.rectangle(image,(x1,y1),(x2,y2),label_color_dict[int(label)],2)
            print(f"{label}:{label_dict[int(label)]}",norm_center_x,norm_center_y,norm_height,norm_height)
        
        
        ## ekranda göster 
        image_ = cv2.resize(image,frame_display_size)
        
        ## bilgi yazıları
        cv2.putText(image_,str(shown_counter),(10,40),font,1.2,color2)
        cv2.putText(image_,str(notproblem_counter),(80,40),font,1.2,color3)    
        cv2.putText(image_,str(problem_counter),(150,40),font,1.2,color4)
    
        for i,text in enumerate(["space:atla","p:problemli","q:bitir"]):
            cv2.putText(image_,text,(10,image_.shape[0]-35*i-25),font,0.8,color2)
        
        window_name = f"{c}  -  {im_name}  -  {label_name}"
    
        cv2.namedWindow(window_name)
        cv2.moveWindow(window_name,20,20)
        cv2.imshow(window_name,image_)
        
        key = cv2.waitKey(0)
        if key == ord("q"):
            cv2.destroyAllWindows()
            clear_output()
            break
        elif key == ord("p"):
            problems.append(f"{c} image: {im_name} txt: {label_name}")
            clear_output()
            cv2.destroyAllWindows()
            
            shown_counter+=1 
            problem_counter +=1
            print(f"Problemli olarak eklendi: {im_name}")
            continue
        elif key == ord(" "):
            cv2.destroyAllWindows()
            clear_output()
            shown_counter+=1 
            notproblem_counter +=1
            continue
            
if len(problems) > 0: 
    q = open("problems.txt","w")
    for problem in problems:
        q.write(dataset_name+" "+problem+"\n")
    q.close()
    f.close()
    print(problems)

3it [04:14, 84.68s/it] 

['train image: 9999955_00000_d_0000284.jpg txt: 9999955_00000_d_0000284.txt', 'train image: 9999987_00000_d_0000006.jpg txt: 9999987_00000_d_0000006.txt', 'train image: 9999955_00000_d_0000121.jpg txt: 9999955_00000_d_0000121.txt', 'train image: 9999999_00271_d_0000118.jpg txt: 9999999_00271_d_0000118.txt', 'train image: 9999964_00000_d_0000074.jpg txt: 9999964_00000_d_0000074.txt', 'train image: 9999982_00000_d_0000215.jpg txt: 9999982_00000_d_0000215.txt', 'train image: 9999955_00000_d_0000302.jpg txt: 9999955_00000_d_0000302.txt', 'train image: 9999956_00000_d_0000054.jpg txt: 9999956_00000_d_0000054.txt', 'train image: 9999955_00000_d_0000079.jpg txt: 9999955_00000_d_0000079.txt', 'train image: 9999966_00000_d_0000071.jpg txt: 9999966_00000_d_0000071.txt', 'train image: 9999945_00000_d_0000148.jpg txt: 9999945_00000_d_0000148.txt', 'train image: 9999955_00000_d_0000352.jpg txt: 9999955_00000_d_0000352.txt', 'train image: 9999953_00000_d_0000141.jpg txt: 9999953_00000_d_0000141.txt'

In [8]:
problems

NameError: name 'problems' is not defined

## Toplanmış veri setinlerinden birini test etmek (./merged altındaki)

In [10]:
ds_list = os.listdir("./merged")
for i,ds in enumerate(ds_list):
    print(i,ds)
    
print("\nIndex girin: ")
d_i = int(input("> "))

if d_i not in range(len(ds_list)):
    print("\nuygun bir index girin")
else:
    
    dataset_name = ds_list[d_i]    
    print("\nseçilen veriseti:",dataset_name)
    
    root = f"./merged/{ds_list[d_i]}/detect"
    print("\nroot: ",root)

    print("\nGörüntüler rastgele gelsin \nhayır-> 0 \nevet-> 1")
    is_rand = int(input("> "))
    clear_output()
    print(dataset_name)
    if is_rand == 0:
        is_randomized = False
        print("Görüntüler rastgele olmayacak")
    elif is_rand ==1:
        is_randomized = True
        print("Görüntüler rastgele olacak")
    else:
        print("Yanlış karakter girildi. Otomatik Tercih: ",is_randomized)

3_Spiral_Vision_Dataset_V4
Görüntüler rastgele olacak


In [11]:
images_path = path_sequence(root,"images")
labels_path = path_sequence(root,"labels")

problems = []

shown_counter = 0
problem_counter = 0
notproblem_counter = 0


for i,c in tqdm( enumerate(["train","test","val"])):
    images_file = path_sequence(images_path,c)
    labels_file = path_sequence(labels_path,c)
    print("işleniyor: ", images_file)
    print("işleniyor: ", labels_file)
    image_names = os.listdir(images_file)
    label_names = os.listdir(labels_file)
    
    if is_randomized:
        image_names, label_names = shuffle(image_names, label_names)
    
    for im_name,label_name in zip(image_names,label_names):
        ## dosyalara ulaş
        print("--",im_name,label_name)
        print("gösterildi: ",shown_counter)
        # frame
        original_image = cv2.imread(path_sequence(images_file,im_name))
        image = original_image.copy()
        
        # etiketler
        f = open(path_sequence(labels_file,label_name),"r")
        lines = f.readlines()
        
        for line in lines:
            label, norm_center_x, norm_center_y, norm_width, norm_height = format_line(line)
            
            width = image.shape[1] * norm_width
            height = image.shape[0] * norm_height
            x1 = int(image.shape[1]*norm_center_x) - int(width/2)
            x2 = int(image.shape[1]*norm_center_x) + int(width/2)
            y1 = int(image.shape[0]*norm_center_y) - int(height/2)
            y2 = int(image.shape[0]*norm_center_y) + int(height/2)
            
            # etiketleri çizdirelim
            cv2.putText(image,f"{label_dict[int(label)]}",(x1+5,y1+15),font,0.55,label_color_dict[int(label)])
            cv2.rectangle(image,(x1,y1),(x2,y2),label_color_dict[int(label)],2)
            print(f"{label}:{label_dict[int(label)]}",norm_center_x,norm_center_y,norm_height,norm_height)
        
        
        ## ekranda göster 
        image_ = cv2.resize(image,frame_display_size)
        
        ## bilgi yazıları
        cv2.putText(image_,str(shown_counter),(10,40),font,1.2,color2)
        cv2.putText(image_,str(notproblem_counter),(80,40),font,1.2,color3)    
        cv2.putText(image_,str(problem_counter),(150,40),font,1.2,color4)
    
        for i,text in enumerate(["space:atla","p:problemli","q:bitir"]):
            cv2.putText(image_,text,(10,image_.shape[0]-35*i-25),font,0.8,color2)
        
        window_name = f"{c}  -  {im_name}  -  {label_name}"
    
        cv2.namedWindow(window_name)
        cv2.moveWindow(window_name,20,20)
        cv2.imshow(window_name,image_)
        
        key = cv2.waitKey(0)
        if key == ord("q"):
            cv2.destroyAllWindows()
            clear_output()
            break
        elif key == ord("p"):
            problems.append(f"{c} image: {im_name} txt: {label_name}")
            clear_output()
            cv2.destroyAllWindows()
            
            shown_counter+=1 
            problem_counter +=1
            print(f"Problemli olarak eklendi: {im_name}")
            continue
        elif key == ord(" "):
            cv2.destroyAllWindows()
            clear_output()
            shown_counter+=1 
            notproblem_counter +=1
            continue
            
if len(problems) > 0: 
    q = open("problems.txt","w")
    for problem in problems:
        q.write(dataset_name+" "+problem+"\n")
    q.close()
    f.close()
    print(problems)

3it [03:17, 65.92s/it]
